# ORIGAMI Pipeline Demo

This notebook demonstrates the full ORIGAMI pipeline:

1. **Build DZI Pyramid** — Take a square, power-of-two source image and tile it into a Deep Zoom Image pyramid
2. **Encode** — `origami encode` computes luma residuals between pyramid levels and packs them
3. **Decode** — `origami decode` reconstructs high-resolution tiles from L2 baseline + residuals
4. **Serve** — `origami serve` does live on-the-fly reconstruction behind an OpenSeadragon viewer

The key insight: instead of storing the full pyramid (~4x storage), we store only L2 tiles + small grayscale residuals, then reconstruct L0/L1 on demand.

## 1. Configuration

In [ ]:
import os
import json
import shutil
import subprocess
import time
import threading
from pathlib import Path
from http.server import HTTPServer, SimpleHTTPRequestHandler

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, HTML, IFrame

# --- Paths ---
SOURCE_IMAGE = "../server/test-data/L0-1024.jpg"  # Square, power-of-two, >= 1024px
WORK_DIR = Path("pipeline_demo_out")

# --- Parameters ---
TILE_SIZE = 256
RESQ = 90          # Residual JPEG quality for encode
DECODE_Q = 90       # Output JPEG quality for decode
SERVE_PORT = 3007   # Port for origami serve
STATIC_PORT = 8765  # Port for static file viewer

# --- Auto-detect origami binary directory ---
BIN_DIR = None
for candidate_dir in [
    Path("../server/target2/release"),
    Path("../server/target2/debug"),
    Path("../server/target/release"),
    Path("../server/target/debug"),
]:
    if (candidate_dir / "origami").is_file():
        BIN_DIR = candidate_dir.resolve()
        break

if BIN_DIR is None:
    raise FileNotFoundError(
        "Could not find origami binary. Build it first:\n"
        "  cd server && cargo build --release\n"
        "Or with CARGO_TARGET_DIR=target2: cargo build"
    )

print(f"Source image:  {SOURCE_IMAGE}")
print(f"Work dir:      {WORK_DIR}")
print(f"Binary dir:    {BIN_DIR}")
print(f"Tile size:     {TILE_SIZE}")
print(f"Residual Q:    {RESQ}")
print(f"Decode Q:      {DECODE_Q}")

## 1b. Encoder Selection

Detect which JPEG encoder backends are available and select one (or all) to run.

| Encoder | Binary | Description |
|---------|--------|-------------|
| **turbojpeg** | `origami` | Default libjpeg-turbo backend (always available) |
| **mozjpeg** | `origami-mozjpeg` | Mozilla's optimized JPEG encoder with trellis quantization |
| **jpegli** | `origami-jpegli` | Google's improved JPEG encoder (~35% better compression) |

In [ ]:
# Detect available encoder backends by probing binaries
# A single binary may support multiple encoders (turbojpeg is always available;
# mozjpeg or jpegli are added via --features at build time).
# mozjpeg and jpegli are mutually exclusive, so if both are needed, separate binaries are required.

import re

def probe_encoders(binary_path):
    """Ask a binary what encoders it supports by triggering the error message."""
    result = subprocess.run(
        [str(binary_path), "encode", "--encoder", "__probe__",
         "--pyramid", "/dev/null", "--out", "/dev/null"],
        capture_output=True, text=True,
    )
    # Parse: "Available: turbojpeg, mozjpeg (...)"
    m = re.search(r"Available:\s*([^(]+)", result.stderr + result.stdout)
    if m:
        return [e.strip() for e in m.group(1).split(",") if e.strip()]
    return ["turbojpeg"]  # fallback

# Scan for origami binaries and collect encoder->binary mapping
ENCODER_MAP = {}  # encoder_name -> binary_path

for binary_name in ["origami", "origami-mozjpeg", "origami-jpegli"]:
    binary_path = BIN_DIR / binary_name
    if binary_path.is_file():
        encoders = probe_encoders(binary_path)
        for enc in encoders:
            if enc not in ENCODER_MAP:
                ENCODER_MAP[enc] = str(binary_path)

print("Available encoders:")
for enc, binary in ENCODER_MAP.items():
    bin_name = Path(binary).name
    print(f"  {enc:12s}  (via {bin_name})")

if not ENCODER_MAP:
    raise RuntimeError("No encoders found!")

# --- SELECT ENCODER(S) ---
# Set to a single encoder name (e.g. "turbojpeg", "mozjpeg") or "all"
SELECTED_ENCODER = "all"

if SELECTED_ENCODER == "all":
    active_encoders = dict(ENCODER_MAP)
    print(f"\nWill run ALL {len(active_encoders)} encoder(s): {', '.join(active_encoders)}")
else:
    if SELECTED_ENCODER not in ENCODER_MAP:
        raise ValueError(f"Encoder '{SELECTED_ENCODER}' not available. Choose from: {list(ENCODER_MAP.keys())}")
    active_encoders = {SELECTED_ENCODER: ENCODER_MAP[SELECTED_ENCODER]}
    print(f"\nSelected encoder: {SELECTED_ENCODER}")

# Default binary for serve (uses first available encoder)
SERVE_ENCODER = list(active_encoders.keys())[0]
ORIGAMI_BIN = active_encoders[SERVE_ENCODER]
print(f"Default binary for serve: {SERVE_ENCODER} ({Path(ORIGAMI_BIN).name})")

## 2. Build DZI Pyramid

Creates a standard Deep Zoom Image pyramid from a square, power-of-two source image by repeated 2x box-filter downsampling.

In [ ]:
def build_pyramid(src_path, out_dir, tile_size=256, jpeg_q=95):
    """Build a DZI tile pyramid from a square power-of-two image."""
    src_path = Path(src_path)
    out_dir = Path(out_dir)

    img = Image.open(src_path).convert("RGB")
    w, h = img.size

    # Validate
    assert w == h, f"Image must be square, got {w}x{h}"
    assert w >= 1024, f"Image must be >= 1024px, got {w}"
    assert (w & (w - 1)) == 0, f"Dimensions must be power-of-two, got {w}"

    # Compute DZI levels: level N has 1x1, level N+k has 2^k x 2^k
    # Deep Zoom convention: level 0 = 1x1, each level doubles
    import math
    max_level = int(math.log2(w))
    # We only need levels down to where a single tile covers the image
    min_level = int(math.log2(tile_size))  # level where image = 1 tile

    files_dir = out_dir / "baseline_pyramid_files"

    total_tiles = 0
    total_bytes = 0
    current = img

    # Generate from highest resolution (max_level) down
    for level in range(max_level, min_level - 1, -1):
        level_size = 2 ** level
        if current.size[0] != level_size:
            current = img.resize((level_size, level_size), Image.LANCZOS)

        level_dir = files_dir / str(level)
        level_dir.mkdir(parents=True, exist_ok=True)

        # Tile this level
        nx = (level_size + tile_size - 1) // tile_size
        ny = (level_size + tile_size - 1) // tile_size

        for ty in range(ny):
            for tx in range(nx):
                left = tx * tile_size
                upper = ty * tile_size
                right = min(left + tile_size, level_size)
                lower = min(upper + tile_size, level_size)
                tile = current.crop((left, upper, right, lower))

                tile_path = level_dir / f"{tx}_{ty}.jpg"
                tile.save(str(tile_path), "JPEG", quality=jpeg_q)
                total_tiles += 1
                total_bytes += tile_path.stat().st_size

        # Downsample for next level
        if level > min_level:
            next_size = level_size // 2
            current = current.resize((next_size, next_size), Image.BOX)

    # Write DZI manifest
    dzi_xml = f"""<?xml version="1.0" encoding="UTF-8"?>
<Image xmlns="http://schemas.microsoft.com/deepzoom/2008"
  Format="jpg"
  Overlap="0"
  TileSize="{tile_size}"
  >
  <Size
    Height="{w}"
    Width="{w}"
  />
</Image>"""
    dzi_path = out_dir / "baseline_pyramid.dzi"
    dzi_path.write_text(dzi_xml)

    print(f"Pyramid built: {out_dir}")
    print(f"  Levels: {min_level}..{max_level} ({max_level - min_level + 1} levels)")
    print(f"  L0 (highest res) = level {max_level} ({2**max_level}x{2**max_level})")
    print(f"  L1 = level {max_level-1}, L2 = level {max_level-2}")
    print(f"  Total tiles: {total_tiles}")
    print(f"  Total size: {total_bytes / 1024:.1f} KB")

    return {
        "max_level": max_level,
        "min_level": min_level,
        "total_tiles": total_tiles,
        "total_bytes": total_bytes,
    }

In [ ]:
# Build the pyramid
PYRAMID_DIR = WORK_DIR / "pyramid"
if PYRAMID_DIR.exists():
    shutil.rmtree(PYRAMID_DIR)

pyramid_info = build_pyramid(SOURCE_IMAGE, PYRAMID_DIR, tile_size=TILE_SIZE)

L0 = pyramid_info["max_level"]
L1 = L0 - 1
L2 = L0 - 2
FILES_DIR = PYRAMID_DIR / "baseline_pyramid_files"

print(f"\nDZI levels: L0={L0}, L1={L1}, L2={L2}")

In [ ]:
# Visualize the pyramid levels
src_img = Image.open(SOURCE_IMAGE)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Source image
axes[0].imshow(src_img)
axes[0].set_title(f"Source ({src_img.size[0]}x{src_img.size[1]})")
axes[0].axis("off")

# L2 tile (single tile covers this level for 1024px source)
l2_tile = Image.open(FILES_DIR / str(L2) / "0_0.jpg")
axes[1].imshow(l2_tile)
axes[1].set_title(f"L2 tile (level {L2}, {l2_tile.size[0]}x{l2_tile.size[1]})")
axes[1].axis("off")

# L1 tiles in a 2x2 grid
l1_mosaic = Image.new("RGB", (TILE_SIZE * 2, TILE_SIZE * 2))
for ty in range(2):
    for tx in range(2):
        tile_path = FILES_DIR / str(L1) / f"{tx}_{ty}.jpg"
        if tile_path.exists():
            l1_mosaic.paste(Image.open(tile_path), (tx * TILE_SIZE, ty * TILE_SIZE))
axes[2].imshow(l1_mosaic)
axes[2].set_title(f"L1 mosaic (level {L1}, 2x2 tiles)")
axes[2].axis("off")

# L0 tiles in a 4x4 grid
l0_mosaic = Image.new("RGB", (TILE_SIZE * 4, TILE_SIZE * 4))
for ty in range(4):
    for tx in range(4):
        tile_path = FILES_DIR / str(L0) / f"{tx}_{ty}.jpg"
        if tile_path.exists():
            l0_mosaic.paste(Image.open(tile_path), (tx * TILE_SIZE, ty * TILE_SIZE))
axes[3].imshow(l0_mosaic)
axes[3].set_title(f"L0 mosaic (level {L0}, 4x4 tiles)")
axes[3].axis("off")

plt.suptitle("DZI Pyramid Levels", fontsize=14)
plt.tight_layout()
plt.show()

## 3. Encode — `origami encode`

The encoder computes luma residuals between predicted (upsampled) and actual tiles at L1 and L0. These grayscale residual images are much smaller than storing full-color tiles.

```
L2 tile → 2x upsample → L1 prediction
L1 actual - L1 prediction → L1 residual (grayscale JPEG)
```

In [ ]:
# Run encode for each active encoder
# Results stored in: WORK_DIR/encoded_{encoder_name}/
encode_results = {}  # encoder_name -> {"dir": Path, "summary": dict, "elapsed": float}

for enc_name, enc_binary in active_encoders.items():
    enc_dir = WORK_DIR / f"encoded_{enc_name}"
    if enc_dir.exists():
        shutil.rmtree(enc_dir)

    cmd = [
        enc_binary, "encode",
        "--pyramid", str(PYRAMID_DIR),
        "--out", str(enc_dir),
        "--resq", str(RESQ),
        "--encoder", enc_name,
        "--pack",
    ]

    print(f"{'='*60}")
    print(f"Encoder: {enc_name}")
    print(f"Running: {' '.join(cmd)}\n")
    t0 = time.time()
    result = subprocess.run(cmd, capture_output=True, text=True)
    elapsed = time.time() - t0

    print(f"Exit code: {result.returncode}  |  Elapsed: {elapsed:.2f}s")
    if result.stderr:
        # Show just the info lines, skip RUST_LOG noise
        for line in result.stderr.strip().split("\n"):
            if "INFO" in line or "ERROR" in line or "WARN" in line:
                print(f"  {line.split('] ')[-1] if '] ' in line else line}")

    if result.returncode != 0:
        print(f"FAILED: {result.stderr}")
        continue

    summary = {}
    summary_path = enc_dir / "summary.json"
    if summary_path.exists():
        with open(summary_path) as f:
            summary = json.load(f)

    encode_results[enc_name] = {
        "dir": enc_dir,
        "summary": summary,
        "elapsed": elapsed,
    }
    print()

# For backward compat with later cells, point ENCODE_DIR to the first encoder
ENCODE_DIR = encode_results[list(encode_results.keys())[0]]["dir"]
print(f"\nDefault ENCODE_DIR (for serve): {ENCODE_DIR}")

In [ ]:
# Compare encode results across encoders
if len(encode_results) > 1:
    print(f"{'Encoder':<12} {'L1 res':>7} {'L0 res':>7} {'Total bytes':>12} {'KB':>8} {'Time':>7}")
    print("-" * 60)
    for enc_name, info in encode_results.items():
        s = info["summary"]
        total_kb = s.get("total_bytes", 0) / 1024
        print(f"{enc_name:<12} {s.get('l1_residuals', '?'):>7} {s.get('l0_residuals', '?'):>7} "
              f"{s.get('total_bytes', '?'):>12} {total_kb:>7.1f} {info['elapsed']:>6.2f}s")

    # Show size comparison
    names = list(encode_results.keys())
    base_bytes = encode_results[names[0]]["summary"].get("total_bytes", 1)
    print(f"\nSize relative to {names[0]}:")
    for enc_name, info in encode_results.items():
        enc_bytes = info["summary"].get("total_bytes", 0)
        ratio = enc_bytes / base_bytes if base_bytes else 0
        savings = (1 - ratio) * 100
        print(f"  {enc_name:<12} {enc_bytes/1024:>7.1f} KB  ({ratio:.2%})"
              + (f"  — {savings:.1f}% smaller" if savings > 0.1 else ""))
else:
    enc_name = list(encode_results.keys())[0]
    s = encode_results[enc_name]["summary"]
    print(f"Encode summary ({enc_name}):")
    for k, v in s.items():
        print(f"  {k}: {v}")

In [ ]:
# Visualize residual tiles (from first encoder)
# Residuals are stored as: {enc_dir}/L1/{x2}_{y2}/{x1}_{y1}.jpg
#                           {enc_dir}/L0/{x2}_{y2}/{x0}_{y0}.jpg

first_enc = list(encode_results.keys())[0]
first_enc_dir = encode_results[first_enc]["dir"]

l1_res_dir = first_enc_dir / "L1"
l0_res_dir = first_enc_dir / "L0"

l1_residuals = []
if l1_res_dir.exists():
    for parent_dir in sorted(l1_res_dir.iterdir()):
        if parent_dir.is_dir():
            for jpg in sorted(parent_dir.glob("*.jpg")):
                l1_residuals.append(jpg)

l0_residuals = []
if l0_res_dir.exists():
    for parent_dir in sorted(l0_res_dir.iterdir()):
        if parent_dir.is_dir():
            for jpg in sorted(parent_dir.glob("*.jpg")):
                l0_residuals.append(jpg)

print(f"Residuals from {first_enc}: L1={len(l1_residuals)}, L0={len(l0_residuals)}")

n_show = min(4, len(l1_residuals))
if n_show > 0:
    fig, axes = plt.subplots(2, n_show, figsize=(4 * n_show, 8))
    if n_show == 1:
        axes = axes.reshape(2, 1)

    for i in range(n_show):
        img = Image.open(l1_residuals[i])
        axes[0, i].imshow(img, cmap="gray", vmin=0, vmax=255)
        axes[0, i].set_title(f"L1 res: {l1_residuals[i].name}")
        axes[0, i].axis("off")

    for i in range(min(n_show, len(l0_residuals))):
        img = Image.open(l0_residuals[i])
        axes[1, i].imshow(img, cmap="gray", vmin=0, vmax=255)
        axes[1, i].set_title(f"L0 res: {l0_residuals[i].name}")
        axes[1, i].axis("off")

    plt.suptitle(f"Residual Tiles — {first_enc} (gray=128 means no difference)", fontsize=14)
    plt.tight_layout()
    plt.show()

# Show pack files
packs_dir = first_enc_dir / "packs"
if packs_dir.exists():
    packs = list(packs_dir.glob("*.pack"))
    total_pack_bytes = sum(p.stat().st_size for p in packs)
    print(f"\nPack files ({first_enc}): {len(packs)}, total {total_pack_bytes / 1024:.1f} KB")
    for p in sorted(packs):
        print(f"  {p.name}: {p.stat().st_size / 1024:.1f} KB")

## 4. Decode — `origami decode`

The decoder reconstructs L0/L1 tiles from the L2 baseline + residuals. This is the inverse of encoding:

```
L2 tile → 2x upsample → L1 prediction
L1 prediction + L1 residual → L1 reconstructed
L1 reconstructed → 2x upsample → L0 prediction
L0 prediction + L0 residual → L0 reconstructed
```

In [ ]:
# Run decode for each encoder's encoded output
# The decode binary is always the default origami (turbojpeg) — it just reads packs
decode_results = {}  # encoder_name -> {"dir": Path, "summary": dict}

decode_binary = ENCODER_MAP.get("turbojpeg", ORIGAMI_BIN)

for enc_name, enc_info in encode_results.items():
    dec_dir = WORK_DIR / f"decoded_{enc_name}"
    if dec_dir.exists():
        shutil.rmtree(dec_dir)

    packs_dir = enc_info["dir"] / "packs"
    cmd = [
        decode_binary, "decode",
        "--pyramid", str(PYRAMID_DIR),
        "--out", str(dec_dir),
        "--packs", str(packs_dir),
        "--quality", str(DECODE_Q),
    ]

    print(f"Decoding {enc_name}...")
    t0 = time.time()
    result = subprocess.run(cmd, capture_output=True, text=True)
    elapsed = time.time() - t0

    if result.returncode != 0:
        print(f"  FAILED: {result.stderr}")
        continue

    summary = {}
    summary_path = dec_dir / "summary.json"
    if summary_path.exists():
        with open(summary_path) as f:
            summary = json.load(f)

    decode_results[enc_name] = {"dir": dec_dir, "summary": summary}
    print(f"  OK: L1={summary.get('l1_tiles', '?')}, L0={summary.get('l0_tiles', '?')}, {elapsed:.2f}s")

# For backward compat
DECODE_DIR = decode_results[list(decode_results.keys())[0]]["dir"]
print(f"\nDefault DECODE_DIR (for serve): {DECODE_DIR}")

In [ ]:
# Show decode summaries
for enc_name, info in decode_results.items():
    s = info["summary"]
    print(f"Decode summary ({enc_name}):")
    for k, v in s.items():
        print(f"  {k}: {v}")
    print()

In [ ]:
# Interactive tile-level quality comparison with linked scroll-wheel zoom
# Layout: for each row of tiles, show baseline row then one reconstructed row per encoder
# All rows are 4 tiles wide. Zoom/pan is linked across ALL tiles.
# Images served from a local HTTP server (no data URIs).

COMPARE_PORT = 8766

class CompareHandler(SimpleHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=str(WORK_DIR), **kwargs)
    def log_message(self, format, *args):
        pass

try:
    compare_server = HTTPServer(("", COMPARE_PORT), CompareHandler)
    compare_thread = threading.Thread(target=compare_server.serve_forever, daemon=True)
    compare_thread.start()
    print(f"Comparison file server on http://localhost:{COMPARE_PORT}")
except OSError:
    print(f"Port {COMPARE_PORT} already in use (server likely already running)")

BASE_URL = f"http://localhost:{COMPARE_PORT}"

# Assign a color to each encoder for visual distinction
ENCODER_COLORS = {
    "turbojpeg": "#d97706",  # amber
    "mozjpeg":   "#059669",  # emerald
    "jpegli":    "#7c3aed",  # violet
}

def fmt_size(path):
    try:
        return f"{path.stat().st_size / 1024:.1f} KB"
    except FileNotFoundError:
        return "?"

def build_comparison_html():
    enc_names = list(decode_results.keys())

    # Build tile data: list of (coord, level, orig_path, {enc: recon_path})
    # L1: 4 tiles
    l1_tiles = []
    for tx in range(2):
        for ty in range(2):
            orig = FILES_DIR / str(L1) / f"{tx}_{ty}.jpg"
            if not orig.exists():
                continue
            recons = {}
            for enc in enc_names:
                r = decode_results[enc]["dir"] / "L1" / f"{tx}_{ty}.jpg"
                if r.exists():
                    recons[enc] = r
            if recons:
                l1_tiles.append((f"{tx},{ty}", L1, orig, recons))

    # L0: rows of 4
    l0_rows = []
    for ty in range(4):
        row = []
        for tx in range(4):
            orig = FILES_DIR / str(L0) / f"{tx}_{ty}.jpg"
            if not orig.exists():
                continue
            recons = {}
            for enc in enc_names:
                r = decode_results[enc]["dir"] / "L0" / f"{tx}_{ty}.jpg"
                if r.exists():
                    recons[enc] = r
            if recons:
                row.append((f"{tx},{ty}", L0, orig, recons))
        if row:
            l0_rows.append(row)

    def tile_cell(url, label_text, color, size_str):
        return f'''<div class="oc-tile-cell">
          <div class="oc-tile-viewport">
            <img src="{url}" draggable="false">
          </div>
          <div class="oc-tile-label" style="color:{color}">
            <span>{label_text}</span>
            <span class="oc-tile-size">{size_str}</span>
          </div>
        </div>'''

    def row_group_html(tiles):
        """For a row of tiles, emit baseline row + one recon row per encoder."""
        n = len(tiles)
        cols = f"grid-template-columns: repeat({n}, 1fr);"
        html = ""
        # Baseline row
        cells = []
        for coord, level, orig, recons in tiles:
            url = f"{BASE_URL}/pyramid/baseline_pyramid_files/{level}/{coord.replace(',', '_')}.jpg"
            cells.append(tile_cell(url, f"Baseline ({coord})", "#2563eb", fmt_size(orig)))
        html += f'<div class="oc-tile-row" style="{cols}">{"".join(cells)}</div>\n'

        # One row per encoder
        for enc in enc_names:
            cells = []
            color = ENCODER_COLORS.get(enc, "#d97706")
            for coord, level, orig, recons in tiles:
                if enc in recons:
                    r = recons[enc]
                    url = f"{BASE_URL}/decoded_{enc}/{('L1' if level == L1 else 'L0')}/{coord.replace(',', '_')}.jpg"
                    cells.append(tile_cell(url, f"{enc} ({coord})", color, fmt_size(r)))
                else:
                    cells.append('<div class="oc-tile-cell"><div class="oc-tile-viewport" style="background:#ddd"></div></div>')
            html += f'<div class="oc-tile-row" style="{cols}">{"".join(cells)}</div>\n'
        return html

    sections_html = ""
    if l1_tiles:
        sections_html += f'''
        <div class="oc-level-section">
          <h3>L1 Tiles (level {L1})</h3>
          {row_group_html(l1_tiles)}
        </div>'''
    if l0_rows:
        l0_html = ""
        for row_tiles in l0_rows:
            l0_html += row_group_html(row_tiles)
        sections_html += f'''
        <div class="oc-level-section">
          <h3>L0 Tiles (level {L0})</h3>
          {l0_html}
        </div>'''

    # Legend items for each encoder
    legend_items = '<span class="oc-legend-baseline" style="color:#2563eb">&#9632; Baseline</span>'
    for enc in enc_names:
        color = ENCODER_COLORS.get(enc, "#d97706")
        legend_items += f'<span style="color:{color}; font-weight:600; font-size:13px">&#9632; {enc}</span>'

    html = f'''
<style>
.origami-compare {{
  font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
  background: #f5f5f5;
  padding: 16px;
  border-radius: 8px;
}}
.origami-compare h2 {{ margin: 0 0 4px 0; color: #333; font-size: 18px; }}
.origami-compare .oc-instructions {{ color: #666; font-size: 13px; margin-bottom: 12px; }}
.oc-toolbar {{
  display: flex; align-items: center; gap: 8px; margin-bottom: 12px; flex-wrap: wrap;
}}
.oc-toolbar button {{
  padding: 4px 12px; font-size: 12px; font-weight: 600;
  border: 1px solid #ccc; border-radius: 4px; background: #fff; cursor: pointer; color: #333;
}}
.oc-toolbar button:hover {{ background: #f0f0f0; }}
.oc-toolbar button.oc-active {{ background: #2563eb; color: #fff; border-color: #2563eb; }}
.oc-zoom-badge {{
  display: inline-block; background: #e0e0e0; color: #333;
  font-size: 12px; font-weight: 600; padding: 3px 10px; border-radius: 10px;
  min-width: 48px; text-align: center;
}}
.origami-compare .oc-legend {{ display: flex; gap: 20px; margin-bottom: 16px; font-size: 13px; font-weight: 600; }}
.oc-level-section {{ margin-bottom: 24px; }}
.oc-level-section h3 {{
  color: #555; font-size: 14px; margin: 0 0 8px 0; padding-bottom: 4px; border-bottom: 1px solid #ddd;
}}
.oc-tile-row {{
  display: grid; grid-template-columns: repeat(4, 1fr); gap: 4px; margin-bottom: 2px;
}}
.oc-tile-cell {{
  background: white; border-radius: 4px; overflow: hidden; border: 1px solid #e0e0e0;
}}
.oc-tile-viewport {{
  width: 100%; aspect-ratio: 1; overflow: hidden; cursor: zoom-in;
  position: relative; background: #eee;
}}
.oc-tile-viewport img {{
  width: 100%; height: 100%; object-fit: fill;
  image-rendering: pixelated; image-rendering: -moz-crisp-edges; image-rendering: crisp-edges;
  transform-origin: center center; transition: none; pointer-events: none; user-select: none;
}}
.oc-tile-label {{
  display: flex; justify-content: space-between;
  font-size: 11px; font-weight: 600; padding: 3px 6px;
  background: #fafafa; border-top: 1px solid #eee;
}}
.oc-tile-size {{ color: #999; font-weight: 400; }}
</style>

<div class="origami-compare">
  <h2>Tile-Level Reconstruction Quality</h2>
  <div class="oc-instructions">Scroll wheel to zoom any tile — all tiles follow. Drag to pan when zoomed. Double-click to reset to fit.</div>
  <div class="oc-toolbar">
    <span class="oc-zoom-badge" id="oc-zoom-indicator">Fit</span>
    <button id="oc-btn-fit">Fit</button>
    <button id="oc-btn-1x">1:1</button>
    <button id="oc-btn-2x">2x</button>
    <button id="oc-btn-4x">4x</button>
    <button id="oc-btn-8x">8x</button>
  </div>
  <div class="oc-legend">
    {legend_items}
  </div>
  ''' + sections_html + f'''
</div>

<script>
(function() {{
  var NATIVE_PX = {TILE_SIZE};
  var viewports = document.querySelectorAll('.oc-tile-viewport');
  var zoomBadge = document.getElementById('oc-zoom-indicator');
  var btnFit = document.getElementById('oc-btn-fit');
  var btn1x = document.getElementById('oc-btn-1x');
  var btn2x = document.getElementById('oc-btn-2x');
  var btn4x = document.getElementById('oc-btn-4x');
  var btn8x = document.getElementById('oc-btn-8x');
  var allBtns = [btnFit, btn1x, btn2x, btn4x, btn8x];

  var shared = {{ scale: 1, panX: 0, panY: 0 }};
  var isPanning = false, startX, startY, startPanX, startPanY;

  function getOneToOneScale() {{
    if (viewports.length === 0) return 1;
    var vpW = viewports[0].getBoundingClientRect().width;
    return NATIVE_PX / vpW;
  }}
  function getMinScale() {{ return Math.min(1, getOneToOneScale()); }}

  function clampPan() {{
    var s = shared.scale;
    if (s <= 1) {{ shared.panX = 0; shared.panY = 0; return; }}
    var maxPan = (s - 1) / (2 * s);
    shared.panX = Math.min(maxPan, Math.max(-maxPan, shared.panX));
    shared.panY = Math.min(maxPan, Math.max(-maxPan, shared.panY));
  }}

  function updateButtons() {{
    allBtns.forEach(function(b) {{ b.classList.remove('oc-active'); }});
    var s = shared.scale, oneToOne = getOneToOneScale();
    if (Math.abs(s - 1) < 0.001) btnFit.classList.add('oc-active');
    else if (Math.abs(s - oneToOne) < 0.001) btn1x.classList.add('oc-active');
    else if (Math.abs(s - oneToOne * 2) < 0.01) btn2x.classList.add('oc-active');
    else if (Math.abs(s - oneToOne * 4) < 0.01) btn4x.classList.add('oc-active');
    else if (Math.abs(s - oneToOne * 8) < 0.01) btn8x.classList.add('oc-active');
  }}

  function applyAll() {{
    var s = shared.scale;
    viewports.forEach(function(vp) {{
      var img = vp.querySelector('img');
      if (!img) return;
      var rect = vp.getBoundingClientRect();
      var px = shared.panX * rect.width * s;
      var py = shared.panY * rect.height * s;
      img.style.transform = 'translate(' + px + 'px, ' + py + 'px) scale(' + s + ')';
      vp.style.cursor = s > 1 ? 'grab' : (s < 1 ? 'default' : 'zoom-in');
    }});
    var oneToOne = getOneToOneScale();
    var displayScale = s / oneToOne;
    if (Math.abs(s - 1) < 0.001) {{
      zoomBadge.textContent = 'Fit';
      zoomBadge.style.background = '#e0e0e0'; zoomBadge.style.color = '#333';
    }} else {{
      zoomBadge.textContent = displayScale.toFixed(1) + 'x';
      zoomBadge.style.background = '#2563eb'; zoomBadge.style.color = '#fff';
    }}
    updateButtons();
  }}

  function setScale(s) {{
    shared.scale = s; shared.panX = 0; shared.panY = 0;
    clampPan(); applyAll();
  }}

  btnFit.addEventListener('click', function() {{ setScale(1); }});
  btn1x.addEventListener('click', function() {{ setScale(getOneToOneScale()); }});
  btn2x.addEventListener('click', function() {{ setScale(getOneToOneScale() * 2); }});
  btn4x.addEventListener('click', function() {{ setScale(getOneToOneScale() * 4); }});
  btn8x.addEventListener('click', function() {{ setScale(getOneToOneScale() * 8); }});

  viewports.forEach(function(vp) {{
    vp.addEventListener('wheel', function(e) {{
      e.preventDefault();
      var rect = vp.getBoundingClientRect();
      var mx = (e.clientX - rect.left) / rect.width - 0.5;
      var my = (e.clientY - rect.top) / rect.height - 0.5;
      var oldScale = shared.scale;
      var factor = e.deltaY > 0 ? 0.8 : 1.25;
      shared.scale = Math.min(16, Math.max(getMinScale(), shared.scale * factor));
      if (shared.scale !== oldScale && shared.scale > 1) {{
        var ratio = shared.scale / oldScale;
        shared.panX = mx / shared.scale - ratio * (mx / shared.scale - shared.panX);
        shared.panY = my / shared.scale - ratio * (my / shared.scale - shared.panY);
      }}
      if (shared.scale <= 1) {{ shared.panX = 0; shared.panY = 0; }}
      clampPan(); applyAll();
    }}, {{passive: false}});

    vp.addEventListener('mousedown', function(e) {{
      if (shared.scale <= 1) return;
      isPanning = true;
      startX = e.clientX; startY = e.clientY;
      startPanX = shared.panX; startPanY = shared.panY;
      e.preventDefault();
      viewports.forEach(function(v) {{ v.style.cursor = 'grabbing'; }});
    }});

    vp.addEventListener('dblclick', function() {{ setScale(1); }});
  }});

  document.addEventListener('mousemove', function(e) {{
    if (!isPanning) return;
    var ref = viewports[0].getBoundingClientRect();
    var dx = (e.clientX - startX) / (ref.width * shared.scale);
    var dy = (e.clientY - startY) / (ref.height * shared.scale);
    shared.panX = startPanX + dx;
    shared.panY = startPanY + dy;
    clampPan(); applyAll();
  }});

  document.addEventListener('mouseup', function() {{
    if (isPanning) {{
      isPanning = false;
      viewports.forEach(function(v) {{
        v.style.cursor = shared.scale > 1 ? 'grab' : (shared.scale < 1 ? 'default' : 'zoom-in');
      }});
    }}
  }});
}})();
</script>
'''
    return html

display(HTML(build_comparison_html()))

## 5. View Decoded Tiles — Static OpenSeadragon Viewer

We assemble a complete DZI pyramid where L0/L1 tiles are replaced by decoded (reconstructed) tiles, then serve it with a simple HTTP server and view it in OpenSeadragon.

In [ ]:
# Build a viewable DZI directory: copy baseline pyramid, overlay decoded L0/L1 tiles
# Uses the first encoder's decoded output
VIEWER_DIR = WORK_DIR / "viewer_static"
if VIEWER_DIR.exists():
    shutil.rmtree(VIEWER_DIR)

shutil.copytree(PYRAMID_DIR, VIEWER_DIR)
viewer_files = VIEWER_DIR / "baseline_pyramid_files"

first_dec = list(decode_results.keys())[0]
first_dec_dir = decode_results[first_dec]["dir"]

l1_decode_dir = first_dec_dir / "L1"
if l1_decode_dir.exists():
    l1_level_dir = viewer_files / str(L1)
    l1_level_dir.mkdir(parents=True, exist_ok=True)
    for tile in l1_decode_dir.glob("*.jpg"):
        shutil.copy2(tile, l1_level_dir / tile.name)

l0_decode_dir = first_dec_dir / "L0"
if l0_decode_dir.exists():
    l0_level_dir = viewer_files / str(L0)
    l0_level_dir.mkdir(parents=True, exist_ok=True)
    for tile in l0_decode_dir.glob("*.jpg"):
        shutil.copy2(tile, l0_level_dir / tile.name)

print(f"Viewer directory assembled: {VIEWER_DIR}")
print(f"Using decoded tiles from: {first_dec}")
print(f"L1 decoded tiles overlaid into level {L1}")
print(f"L0 decoded tiles overlaid into level {L0}")

In [ ]:
# Start a background HTTP server for the static viewer
class QuietHandler(SimpleHTTPRequestHandler):
    """HTTP handler that serves from VIEWER_DIR and suppresses logs."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=str(VIEWER_DIR), **kwargs)

    def log_message(self, format, *args):
        pass  # Suppress request logs in notebook output

static_server = HTTPServer(("", STATIC_PORT), QuietHandler)
static_thread = threading.Thread(target=static_server.serve_forever, daemon=True)
static_thread.start()

print(f"Static file server running on http://localhost:{STATIC_PORT}")

In [ ]:
# Display OpenSeadragon viewer for the decoded tiles
viewer_html = f"""
<div style="border: 1px solid #ccc; border-radius: 4px; overflow: hidden;">
  <div id="osd-static" style="width: 100%; height: 500px;"></div>
</div>
<script src="https://cdnjs.cloudflare.com/ajax/libs/openseadragon/4.1.0/openseadragon.min.js"></script>
<script>
OpenSeadragon({{
  id: "osd-static",
  prefixUrl: "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/4.1.0/images/",
  showNavigator: true,
  tileSources: {{
    Image: {{
      xmlns: "http://schemas.microsoft.com/deepzoom/2008",
      Url: "http://localhost:{STATIC_PORT}/baseline_pyramid_files/",
      Format: "jpg",
      Overlap: "0",
      TileSize: "{TILE_SIZE}",
      Size: {{ Width: "{2**L0}", Height: "{2**L0}" }}
    }}
  }},
  animationTime: 0.6,
  maxZoomPixelRatio: 2,
  zoomPerScroll: 1.2
}});
</script>
"""

display(HTML("<h3>Decoded Tiles Viewer (static)</h3>"))
display(HTML(viewer_html))

## 6. Live Server — `origami serve`

The `origami serve` command does everything the decode step does, but **on-the-fly** per request. It only needs:
- L2 baseline tiles (the smallest pyramid level with full coverage)
- Residual pack files

When a client requests an L0 or L1 tile, the server reconstructs it from L2 + residuals, caches the result, and serves it. This is how ORIGAMI achieves ~5-6x storage reduction with transparent tile serving.

In [ ]:
# Set up the slide directory structure that origami serve expects
# Uses the first encoder's pack files
SLIDES_DIR = WORK_DIR / "slides"
SLIDE_ID = "demo"
SLIDE_DIR = SLIDES_DIR / SLIDE_ID

if SLIDES_DIR.exists():
    shutil.rmtree(SLIDES_DIR)

SLIDE_DIR.mkdir(parents=True)

first_enc = list(encode_results.keys())[0]
first_enc_dir = encode_results[first_enc]["dir"]

(SLIDE_DIR / "baseline_pyramid.dzi").symlink_to((PYRAMID_DIR / "baseline_pyramid.dzi").resolve())
(SLIDE_DIR / "baseline_pyramid_files").symlink_to((PYRAMID_DIR / "baseline_pyramid_files").resolve())
(SLIDE_DIR / "residual_packs").symlink_to((first_enc_dir / "packs").resolve())

print(f"Slide directory: {SLIDE_DIR}")
print(f"  Using packs from: {first_enc}")
print(f"  baseline_pyramid.dzi -> {PYRAMID_DIR / 'baseline_pyramid.dzi'}")
print(f"  baseline_pyramid_files/ -> {PYRAMID_DIR / 'baseline_pyramid_files'}")
print(f"  residual_packs/ -> {first_enc_dir / 'packs'}")

In [ ]:
# Start origami serve as a background process
import signal

serve_cmd = [
    ORIGAMI_BIN, "serve",
    "--slides-root", str(SLIDES_DIR),
    "--port", str(SERVE_PORT),
    "--tile-quality", str(DECODE_Q),
]

print(f"Starting: {' '.join(serve_cmd)}")
serve_proc = subprocess.Popen(
    serve_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Wait for the server to be ready
import urllib.request

ready = False
for attempt in range(30):
    try:
        resp = urllib.request.urlopen(f"http://localhost:{SERVE_PORT}/healthz", timeout=1)
        if resp.status == 200:
            ready = True
            break
    except Exception:
        pass
    time.sleep(0.5)

if ready:
    print(f"Server ready on http://localhost:{SERVE_PORT}")
    print(f"Viewer URL: http://localhost:{SERVE_PORT}/viewer/{SLIDE_ID}")
else:
    print("WARNING: Server did not become ready within 15 seconds")
    if serve_proc.poll() is not None:
        print(f"Process exited with code {serve_proc.returncode}")
        print(f"stderr: {serve_proc.stderr.read().decode()}")

In [ ]:
# Display the live server viewer in an iframe
if ready:
    display(HTML(f"<h3>Live Server Viewer (origami serve)</h3>"))
    display(IFrame(f"http://localhost:{SERVE_PORT}/viewer/{SLIDE_ID}", width=900, height=600))
else:
    print("Server not running — skipping viewer")

## 7. Cleanup

Stop background servers and optionally remove the working directory.

In [ ]:
# Stop the origami serve process
if 'serve_proc' in dir() and serve_proc.poll() is None:
    serve_proc.terminate()
    serve_proc.wait(timeout=5)
    print("origami serve stopped")
else:
    print("origami serve was not running")

# Stop the static file server
if 'static_server' in dir():
    static_server.shutdown()
    print("Static file server stopped")

# Stop the comparison file server
if 'compare_server' in dir():
    compare_server.shutdown()
    print("Comparison file server stopped")

In [ ]:
# Uncomment to remove the working directory
# shutil.rmtree(WORK_DIR)
# print(f"Removed {WORK_DIR}")